# creating data


In [2]:
!pip install -U sentence-transformers 
!pip install pandas

In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
from tqdm import tqdm
import ast
import pandas as pd
import numpy as np
import itertools
# load existing data 
try:
  df = pd.read_csv("./database/disease_to_acu_and_symptom_modify_ZH_faceonly_embedding.csv")
  df['symptoms'] = df['symptoms'].apply(lambda x: ast.literal_eval(x))
  df['acupoints'] = df['acupoints'].apply(lambda x: ast.literal_eval(x))
  df['symptoms_embeddings'] = df['symptoms_embeddings'].apply(lambda x: ast.literal_eval(x))
  print(type(df.loc[0,'symptoms']), type(df.loc[0,'acupoints']),type(df.loc[0,'symptoms_embeddings']),type(df.loc[0,'symptoms_embeddings'][0]))
  print(df.head(5))
  embedder = SentenceTransformer('bert-base-chinese')
except:
  print('no existing data ')

In [ ]:
# create symptoms _embeddings
embedder = SentenceTransformer('bert-base-chinese')

df['symptoms_embeddings'] = [[None]] * len(df)
for index, row in tqdm(df.iterrows()):
  df.loc[index,'symptoms_embeddings'] = embedder.encode(df.loc[index,'symptoms'])

## caculate embedding and save to csV


In [ ]:
df1 = df.copy()
# change np array to list (for saving)
for index, row in df1.iterrows():
  df1.loc[index,'symptoms_embeddings'] = df1.loc[index,'symptoms_embeddings'].tolist()
df1.to_csv('./database/disease_to_acu_and_symptom_modify_ZH_faceonly_embedding.csv',index=False)
df1

# read embedding file and predict


In [10]:
# to use the service, server need to 'pip install -U sentence-transformers '
from sentence_transformers import SentenceTransformer
import scipy.spatial
from tqdm import tqdm
import ast
import pandas as pd
import numpy as np
import itertools

def read_embedding_CSV(file_name = "disease_to_acu_and_symptom_modify_ZH_faceonly_embedding.csv"):
    df = pd.read_csv(file_name)
    df['symptoms'] = df['symptoms'].apply(lambda x: ast.literal_eval(x))
    df['acupoints'] = df['acupoints'].apply(lambda x: ast.literal_eval(x))
    df['symptoms_embeddings'] = df['symptoms_embeddings'].apply(lambda x: ast.literal_eval(x))
    # print(type(df.loc[0,'symptoms']), type(df.loc[0,'acupoints']),type(df.loc[0,'symptoms_embeddings']),type(df.loc[0,'symptoms_embeddings'][0]))
    # print(df.head(5))
    return df

def load_embedder():
    embedder = SentenceTransformer('bert-base-chinese')
    return embedder

df = read_embedding_CSV()
embedder = load_embedder()


Some weights of the model checkpoint at C:\Users\tsoudibi/.cache\torch\sentence_transformers\bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:


def query_disease(queries):
    query_embeddings = embedder.encode(queries)
    threshold = 0.80
    
    # create empty disease dictionary
    score_dist = {} 
    for index, row in df.iterrows():
        score_dist[row['disease_name']] = 0 # use dictionary to store disease name and score

    for query, query_embedding in zip(queries, query_embeddings):
        for index, row in df.iterrows():
            dists = scipy.spatial.distance.cdist([query_embedding], df.loc[index,'symptoms_embeddings'], "cosine")[0]

            final_dist = 0
            avaliable_count = 0
            for dist in dists:
                if 1-dist >= threshold:
                    final_dist += dist
                    avaliable_count += 1
            # take mean value of all diatence as score_dist
            try:
                score_dist[row['disease_name']] = score_dist[row['disease_name']] + (1-(final_dist / avaliable_count))
                # score_dist.append(final_dist / avaliable_count)
            except:
                score_dist[row['disease_name']] = score_dist[row['disease_name']] + 0

        # sort by distence
        results = sorted(score_dist.items(), key=lambda x:x[1], reverse=True)
    print("======================")
    print("Query:", queries)
    for name, distance in results[0:3]:
        print(name, "(Score: %.4f)" % (distance))
    
    # returning top 3 high result
    return results[0:3]

queries = ['眼睛癢','眼窩痛',]
query_disease(queries)

Query: ['眼睛癢', '眼窩痛']
眼眶脹痛 (Score: 1.7586)
眶上神經痛 (Score: 1.7412)
眼內或周圍疼痛 (Score: 1.7233)


## combine to a class

In [15]:
# to use the service, server need to 'pip install -U sentence-transformers '
from sentence_transformers import SentenceTransformer
import scipy.spatial
from tqdm import tqdm
import ast
import pandas as pd
import numpy as np
import itertools

class Medical_Bot():
        
    def read_embedding_CSV(self):
        try:
            df = pd.read_csv(self.DB_file_name)
            df['symptoms'] = df['symptoms'].apply(lambda x: ast.literal_eval(x))
            df['acupoints'] = df['acupoints'].apply(lambda x: ast.literal_eval(x))
            df['symptoms_embeddings'] = df['symptoms_embeddings'].apply(lambda x: ast.literal_eval(x))
            # print(type(df.loc[0,'symptoms']), type(df.loc[0,'acupoints']),type(df.loc[0,'symptoms_embeddings']),type(df.loc[0,'symptoms_embeddings'][0]))
            # print(df.head(5))
            self.DataBase = df
            return 
        except:
            print('error when read_embedding_CSV')

            
    def query_disease(self, queries):
        # encode quries
        query_embeddings = embedder.encode(queries)
        
        # create empty disease dictionary
        score_dist = {} 
        for index, row in df.iterrows():
            score_dist[row['disease_name']] = 0 # use dictionary to store disease name and score

        for query, query_embedding in zip(queries, query_embeddings):
            for index, row in df.iterrows():
                dists = scipy.spatial.distance.cdist([query_embedding], df.loc[index,'symptoms_embeddings'], "cosine")[0]

                final_dist = 0
                avaliable_count = 0
                for dist in dists:
                    if 1-dist >= self.query_threshold :
                        final_dist += dist
                        avaliable_count += 1
                # take mean value of all diatence as score_dist
                try:
                    score_dist[row['disease_name']] = score_dist[row['disease_name']] + (1-(final_dist / avaliable_count))
                    # score_dist.append(final_dist / avaliable_count)
                except:
                    score_dist[row['disease_name']] = score_dist[row['disease_name']] + 0

            # sort by distence
            results = sorted(score_dist.items(), key=lambda x:x[1], reverse=True)
        print("======================")
        print("Query:", queries)
        for name, distance in results[0:3]:
            print(name, "(Score: %.4f)" % (distance))
        
        # returning top 3 high result
        return results[0:self.query_return_number]
            
    def __init__(self):        
        # DB settings
        print('reading database file...')
        self.DB_file_name = "disease_to_acu_and_symptom_modify_ZH_faceonly_embedding.csv"
        self.read_embedding_CSV()
        print('done!')
        
        # load embedder
        print('loading embedder...')
        self.embedder = SentenceTransformer('bert-base-chinese')
        print('done!')
        
        # query settings
        self.query_return_number = 3
        self.query_threshold = 0.80


In [16]:
MB = Medical_Bot()


reading database file...
done!
loading embedder...


Some weights of the model checkpoint at C:\Users\tsoudibi/.cache\torch\sentence_transformers\bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done!


In [17]:
MB.query_disease(['眼睛癢','眼窩痛',])

Query: ['眼睛癢', '眼窩痛']
眼眶脹痛 (Score: 1.7586)
眶上神經痛 (Score: 1.7412)
眼內或周圍疼痛 (Score: 1.7233)


[('眼眶脹痛', 1.758614956663484),
 ('眶上神經痛', 1.7412440078948541),
 ('眼內或周圍疼痛', 1.7232594403991994)]

正反慈


In [1]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
from tqdm import tqdm
import ast
import pandas as pd
import numpy as np
import itertools

In [ ]:
embedder = SentenceTransformer('bert-base-chinese')

In [26]:
AA = '會'
BB = '有'
A = embedder.encode(AA)
B = embedder.encode(BB)
dists = scipy.spatial.distance.cdist([A], [B], "cosine")[0]
print(AA,' ',BB,' :', dists)

ValueError: XA must be a 2-dimensional array.

In [ ]:
import seaborn as sns